In [1]:
## All imports needed
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import urllib.request
import numpy as np
import pandas as pd
import torch
from torch import nn

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

Load the data

In [16]:
PATH = '../data/nietzsche/'

os.makedirs(PATH, exist_ok=True)
os.makedirs(f'{PATH}trn', exist_ok=True)
os.makedirs(f'{PATH}vld', exist_ok=True)

urllib.request.urlretrieve("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')

# Put first 80% of text into trn and the rest into vld
wc = !wc -l {PATH}nietzsche.txt
wc = int(wc[0].split(' ')[0])

trn_n = int(round(wc*0.8))
vld_n = wc - trn_n

!head -n {trn_n} {PATH}nietzsche.txt > {PATH}trn/text.txt
!tail -n {vld_n} {PATH}nietzsche.txt > {PATH}vld/text.txt

Build out model

In [17]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

md = LanguageModelData.from_text_files(PATH, TEXT, train='trn', validation='vld', test='vld', bs=bs, bptt=bptt, min_freq=3)